In [1]:
import wandb
import click
import torch
import transformers
from pathlib import Path
from copy import deepcopy
from functools import partial
from typing import List, Callable, Dict, Iterable

# Local imports
try:
    import _pathfix
except ImportError:
    from . import _pathfix
from loops import training_loop
from config import LOCATIONS as LOC, CONFIG
from models.multitask import BasicMTL
from dataiter import MultiTaskDataIter
# from eval import ner_all, ner_only_annotated, ner_span_recog_recall, ner_span_recog_precision, \
#     pruner_p, pruner_r


Fixing paths from /home/priyansh/Dev/research/coref/mtl/src


In [2]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:            15G        8,7G        1,6G        1,0G        5,1G        5,3G
Swap:          979M        555M        424M


In [3]:

def make_optimizer(model, optimizer_class: Callable, lr: float, freeze_encoder: bool):
    if freeze_encoder:
        return optimizer_class(
            [param for name, param in model.named_parameters() if not name.startswith("encoder")],
            lr=lr
        )
    else:
        return optimizer_class(model.parameters(), lr=lr)


def get_pretrained_dirs(nm: str):
    """Check if the given nm is stored locally. If so, load that. Else, pass it on as is."""
    plausible_parent_dir: Path = LOC.root / "models" / "huggingface" / nm

    if (
            (plausible_parent_dir / "config").exists()
            and (plausible_parent_dir / "tokenizer").exists()
            and (plausible_parent_dir / "encoder").exists()
    ):
        return (
            str(plausible_parent_dir / "config"),
            str(plausible_parent_dir / "tokenizer"),
            str(plausible_parent_dir / "encoder"),
        )
    else:
        return nm, nm, nm


def compute_metrics(metrics: Dict[str, Callable], logits, labels) -> Dict[str, float]:
    return {metric_nm: metric_fn(logits=logits, labels=labels).item() for metric_nm, metric_fn in metrics.items()}


def aggregate_metrics(inter_epoch: dict, intra_epoch: dict):
    for task_nm in inter_epoch.keys():
        for metric_nm, metric_list in intra_epoch[task_nm].items():
            inter_epoch[task_nm][metric_nm].append(np.mean(metric_list))
    return inter_epoch


def simplest_loop(
        epochs: int,
        tasks: Iterable[str],
        opt: torch.optim,
        train_fn: Callable,
        predict_fn: Callable,
        trn_dl: Callable,
        dev_dl: Callable,
        eval_fns: dict,
) -> (list, list, list):
    train_loss = {task_nm: [] for task_nm in tasks}
    train_metrics = {task_nm: {metric_nm: [] for metric_nm in eval_fns[task_nm].keys()} for task_nm in tasks}
    valid_metrics = {task_nm: {metric_nm: [] for metric_nm in eval_fns[task_nm].keys()} for task_nm in tasks}

    # Make data
    trn_ds = trn_dl()
    dev_ds = dev_dl()

    # Epoch level
    for e in range(epochs):

        per_epoch_loss = {task_nm: [] for task_nm in tasks}
        per_epoch_tr_metrics = {task_nm: {metric_nm: [] for metric_nm in eval_fns[task_nm].keys()} for task_nm in tasks}
        per_epoch_vl_metrics = {task_nm: {metric_nm: [] for metric_nm in eval_fns[task_nm].keys()} for task_nm in tasks}

        # Train
        with Timer() as timer:

            # Train Loop
            for instance in tqdm(trn_ds):

                # Reset the gradients.
                opt.zero_grad()

                # Forward Pass
                outputs = train_fn(**instance)

                """
                    Depending on instance.tasks list, do the following:
                        - task specific loss (added to losses)
                        - task specific metrics (added to metrics)
                """
                for task_nm in instance['tasks']:
                    loss = outputs["loss"][task_nm]
                    per_epoch_loss[task_nm].append(loss.item())

                    # TODO: add other metrics here
                    instance_metrics = compute_metrics(eval_fns[task_nm],
                                                       logits=outputs[task_nm]["logits"],
                                                       labels=outputs[task_nm]["labels"])
                    for metric_nm, metric_vl in instance_metrics.items():
                        per_epoch_tr_metrics[task_nm][metric_nm].append(metric_vl)

                loss.backward()
                opt.step()

            # Val
            with torch.no_grad():

                for instance in tqdm(dev_ds):
                    outputs = predict_fn(**instance)

                    for task_nm in instance["tasks"]:
                        logits = outputs[task_nm]["logits"]
                        # TODO: make the label puller task specific somehow
                        labels = instance["ner"]["gold_labels"]

                        instance_metrics = compute_metrics(eval_fns[task_nm], logits=logits, labels=labels)
                        for metric_nm, metric_vl in instance_metrics.items():
                            per_epoch_vl_metrics[task_nm][metric_nm].append(metric_vl)

        # Bookkeep
        for task_nm in tasks:
            train_loss[task_nm].append(np.mean(per_epoch_loss[task_nm]))
            train_metrics = aggregate_metrics(train_metrics, per_epoch_tr_metrics)
            valid_metrics = aggregate_metrics(valid_metrics, per_epoch_vl_metrics)

        print(f"\nEpoch: {e:3d}" +
              ''.join([f" | {task_nm} Loss: {float(np.mean(per_epoch_loss[task_nm])):.5f}" +
                       ''.join([f" | {task_nm} Tr_{metric_nm}: {float(metric_vls[-1]):.3f}"
                                for metric_nm, metric_vls in train_metrics[task_nm].items()]) +
                       ''.join([f" | {task_nm} Vl_{metric_nm}: {float(metric_vls[-1]):.3f}"
                                for metric_nm, metric_vls in valid_metrics[task_nm].items()])
                       # f" | {task_nm} Tr_c: {float(np.mean(per_epoch_tr_acc[task_nm])):.5f}" +
                       # f" | {task_nm} Vl_c: {float(np.mean(per_epoch_vl_acc[task_nm])):.5f}"
                       for task_nm in tasks]))

    return train_metrics, valid_metrics, train_loss

# Make MTL A

In [18]:
dataset: str = 'ontonotes'
epochs: int = 10
encoder: str = "bert-base-uncased"
tasks: List[str] = ('coref', 'ner', 'pruner')
device: str = "cpu"
trim: bool = False
train_encoder: bool = False,
ner_unweighted: bool = False
filter_candidates_pos = True

In [19]:

dir_config, dir_tokenizer, dir_encoder = get_pretrained_dirs(encoder)

tokenizer = transformers.BertTokenizer.from_pretrained(dir_tokenizer)
config = transformers.BertConfig(dir_config)
config.max_span_width = 5
config.coref_dropout = 0.3
config.metadata_feature_size = 20
config.unary_hdim = 1000
config.binary_hdim = 2000
config.top_span_ratio = 0.4
config.max_top_antecedents = 50
config.device = device
config.epochs = epochs
config.trim = trim
config.freeze_encoder = not train_encoder
config.ner_ignore_weights = ner_unweighted
config.filter_candidates_pos_threshold = CONFIG['filter_candidates_pos_threshold'] \
    if filter_candidates_pos else -1


if 'ner' in tasks or 'pruner' in tasks:
    # Need to figure out the number of classes. Load a DL. Get the number. Delete the DL.
    temp_ds = MultiTaskDataIter(
        src=dataset,
        config=config,
        tasks=tasks,
        split="development",
        tokenizer=tokenizer,
    )
    if 'ner' in tasks:
        config.ner_n_classes = deepcopy(temp_ds.ner_tag_dict.__len__())
        config.ner_class_weights = temp_ds.estimate_class_weights('ner')
    else:
        config.ner_n_classes = 1
        config.ner_class_weights = [1.0, ]
    if 'pruner' in tasks:
        config.pruner_class_weights = temp_ds.estimate_class_weights('pruner')
    del temp_ds
else:
    config.ner_n_classes = 1
    config.ner_class_weights = [1.0, ]

# # Make the model
# model = BasicMTL(dir_encoder, config=config)

# Load the data
train_ds = partial(
    MultiTaskDataIter,
    src=dataset,
    config=config,
    tasks=tasks,
    split="train",
    tokenizer=tokenizer,
)
# valid_ds = partial(
#     MultiTaskDataIter,
#     src=dataset,
#     config=config,
#     tasks=tasks,
#     split="development",
#     tokenizer=tokenizer,
# )

# Make the optimizer
# opt = make_optimizer(model=model, optimizer_class=torch.optim.SGD, lr=0.005, freeze_encoder=config.freeze_encoder)
# opt = torch.optim.SGD(model.parameters(), lr=0.001)

# # Make the evaluation suite (may compute multiple metrics corresponding to the tasks)
# eval_fns: Dict[str, Dict[str, Callable]] = {
#     'ner': {'acc': ner_all,
#             'acc_l': ner_only_annotated,
#             'span_p': ner_span_recog_precision,
#             'span_r': ner_span_recog_recall},
#     'coref': {

#     },
#     'pruner': {'p': pruner_p,
#                'r': pruner_r}
# }

print(config)
print("Training commences!")

/home/priyansh/Dev/research/coref/mtl/src/dataiter.py:164: UserWarning: Processed (training ready) found on ../data/parsed/ontonotes/development/MultiTaskDatasetDump_coref_ner_pruner.pkl. But the config files mismatch.Reprocessing will commence now but will take some time. Approx. 5 min.
  warnings.warn(


Token indices sequence length is longer than the specified maximum sequence length for this model (1083 > 512). Running this sequence through the model will result in indexing errors


TOO LONG:  reeeeeeeeeeeeeeaaaal
TOO LONG:  586.14_588.20_
TOO LONG:  586.14_588.20_a
TOO LONG:  586.14_588.20_a:
TOO LONG:  820.26_822.65_b
TOO LONG:  820.26_822.65_b:
TOO LONG:  965.04_967.89_
TOO LONG:  965.04_967.89_b
TOO LONG:  965.04_967.89_b:

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "binary_hdim": 2000,
  "classifier_dropout": null,
  "coref_dropout": 0.3,
  "device": "cpu",
  "epochs": 10,
  "filter_candidates_pos_threshold": 2000,
  "freeze_encoder": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "max_span_width": 5,
  "max_top_antecedents": 50,
  "metadata_feature_size": 20,
  "model_type": "bert",
  "ner_class_weights": [
    0.05347951180430152,
    18.464160137323844,
    41.44081277574101,
    22.342691547455527,
    15.538011695906432,
    23.87780164697458,
    178.17445899011489,
    626.7922932330827,

In [20]:
onto_train_di = train_ds()

/home/priyansh/Dev/research/coref/mtl/src/dataiter.py:164: UserWarning: Processed (training ready) found on ../data/parsed/ontonotes/train/MultiTaskDatasetDump_coref_ner_pruner.pkl. But the config files mismatch.Reprocessing will commence now but will take some time. Approx. 5 min.
  warnings.warn(


TOO LONG:  xzx620521.blogc
TOO LONG:  xzx620521.blogcn
TOO LONG:  xzx620521.blogcn.
TOO LONG:  xzx...@sina.
TOO LONG:  -lrb-c-rrb-?
TOO LONG:  -lrb-c-rrb-?io
TOO LONG:  -lrb-c-rrb-?io]
TOO LONG:  -lrb-c-rrb-?io]o
TOO LONG:  -lrb-c-rrb-?io]o?
TOO LONG:  -lrb-c-rrb-x
TOO LONG:  io...-lrb-c-
TOO LONG:  io...-lrb-c-rr
TOO LONG:  io...-lrb-c-rrb
TOO LONG:  io...-lrb-c-rrb-
TOO LONG:  io...-lrb-c-rrb-x
TOO LONG:  io...-lrb-c-rrb-x.
TOO LONG:  io...-lrb-c-rrb-x.go
TOO LONG:  io...-lrb-c-rrb-x.goio
TOO LONG:  io...-lrb-c-rrb-x.goio]
TOO LONG:  339.00_347.11_a
TOO LONG:  339.00_347.11_a:
TOO LONG:  263.85_264.42_a:
TOO LONG:  1622.82_1623.11_b:
TOO LONG:  1685.48_1686.56_b
TOO LONG:  1685.48_1686.56_b:
TOO LONG:  114.86_118.28_a:
TOO LONG:  387.98_388.72_
TOO LONG:  387.98_388.72_a
TOO LONG:  387.98_388.72_a:
TOO LONG:  437.08_438.30_
TOO LONG:  437.08_438.30_b
TOO LONG:  437.08_438.30_b:
TOO LONG:  685.61_687.71_
TOO LONG:  685.61_687.71_a
TOO LONG:  685.61_687.71_a:
TOO LONG:  458.93_461.27_


# Make MTL B

In [21]:
dataset: str = 'scierc'
epochs: int = 10
encoder: str = "bert-base-uncased"
tasks: List[str] = ('ner',)
device: str = "cpu"
trim: bool = False
train_encoder: bool = False
ner_unweighted: bool = False
filter_candidates_pos = True


dir_config, dir_tokenizer, dir_encoder = get_pretrained_dirs(encoder)

tokenizer = transformers.BertTokenizer.from_pretrained(dir_tokenizer)
config = transformers.BertConfig(dir_config)
config.max_span_width = 5
config.coref_dropout = 0.3
config.metadata_feature_size = 20
config.unary_hdim = 1000
config.binary_hdim = 2000
config.top_span_ratio = 0.4
config.max_top_antecedents = 50
config.device = device
config.epochs = epochs
config.trim = trim
config.freeze_encoder = not train_encoder
config.ner_ignore_weights = ner_unweighted
config.filter_candidates_pos_threshold = CONFIG['filter_candidates_pos_threshold'] \
    if filter_candidates_pos else -1


# if 'ner' in tasks or 'pruner' in tasks:
if False:
    # Need to figure out the number of classes. Load a DL. Get the number. Delete the DL.
    temp_ds = MultiTaskDataIter(
        src=dataset,
        config=config,
        tasks=tasks,
        split="dev",
        tokenizer=tokenizer,
    )
    if 'ner' in tasks:
        config.ner_n_classes = deepcopy(temp_ds.ner_tag_dict.__len__())
        config.ner_class_weights = temp_ds.estimate_class_weights('ner')
    else:
        config.ner_n_classes = 1
        config.ner_class_weights = [1.0, ]
    if 'pruner' in tasks:
        config.pruner_class_weights = temp_ds.estimate_class_weights('pruner')
    del temp_ds
else:
    config.ner_n_classes = 1
    config.ner_class_weights = [1.0, ]

# # Make the model
# model = BasicMTL(dir_encoder, config=config)

# Load the data
train_ds = partial(
    MultiTaskDataIter,
    src=dataset,
    config=config,
    tasks=tasks,
    split="train",
    tokenizer=tokenizer,
)

sci_train_ds = train_ds()

/home/priyansh/Dev/research/coref/mtl/src/dataiter.py:164: UserWarning: Processed (training ready) found on ../data/parsed/scierc/train/MultiTaskDatasetDump_ner.pkl. But the config files mismatch.Reprocessing will commence now but will take some time. Approx. 5 min.
  warnings.warn(


In [22]:
sci_train_ds[2] 

{'tasks': ['ner'],
 'input_ids': tensor([[ 1999,  2023,  3259,  1010,  2057,  2556,  1037,  3617,  4742, 13151,
           1006, 16233,  2361,  1007,  7375,  1997,  2613,  1011,  2051,  7778,
           2376,  7584,  1006, 18315,  1007,  2005,  4333,  4613, 22415,  1998,
          16175, 28221, 15465,  2389,  4613, 22415,  1012,  2004,  1037,  4333,
           4613,  8278,  1010,  2057,  3579,  2006,  2512,  1011, 19525, 20227,
           1006, 15125,  1007,  1010,  2029,  2064,  2022,  2109,  1999,  8146,
           2073, 19525,  4613,  2003,  2025, 11701,  1012, 16175, 28221, 15465,
           2389,  4613,  2003,  2028,  1997,  1996,  5171,  4127,  1997, 21862,
          18143,  3351,  2389,  4613,  2550,  2011,  2019,  4522,  4092,  4118,
           2005,  2474, 18143,  3351,  6593,  8462,  2229,  1012,  2174,  1010,
           1996,  2614,  3737,  1997, 15125,  1998, 16175, 28221, 15465,  2389,
           4613, 17567,  2013,  3768,  1997,  3019,  2791,  1012, 18315,  2038,
        

In [38]:
la = onto_train_di.__len__()
lb = sci_train_ds.__len__()

la, lb, la+lb

(2455, 346, 2801)

In [31]:
sampling_ratio = [1, 1]

In [32]:
llen = la + lb

In [37]:
pointers = [int(x* llen / float(sum(sampling_ratio))) for x in sampling_ratio]
pointers

[1400, 1400]

# Testing Eval

In [ ]:
from eval import Evaluator, NERAcc, NERSpanRecognitionPR, PrunerPR, CorefBCubed, CorefMUC, CorefCeafe

In [ ]:
eval_bench = Evaluator(
    predict_fn = model.pred_with_labels,
    dataset_partial = valid_ds,
    metrics = [NERAcc(), NERSpanRecognitionPR(), PrunerPR(), CorefBCubed(), CorefMUC(), CorefCeafe()],
    device = 'cpu'
)

In [ ]:
eval_bench.run()

# Eval for Coref

In [ ]:


def b_cubed(clusters, mention_to_gold):
    num, dem = 0, 0

    for c in clusters:
        if len(c) == 1:
            continue

        gold_counts = Counter()
        correct = 0
        for m in c:
            if m in mention_to_gold:
                gold_counts[tuple(mention_to_gold[m])] += 1
        for c2, count in gold_counts.items():
            if len(c2) != 1:
                correct += count * count

        num += correct / float(len(c))
        dem += len(c)

    return num, dem


def muc(clusters, mention_to_gold):
    tp, p = 0, 0
    for c in clusters:
        p += len(c) - 1
        tp += len(c)
        linked = set()
        for m in c:
            if m in mention_to_gold:
                linked.add(mention_to_gold[m])
            else:
                tp -= 1
        tp -= len(linked)
    return tp, p


def phi4(c1, c2):
    return 2 * len([m for m in c1 if m in c2]) / float(len(c1) + len(c2))


def ceafe(clusters, gold_clusters):
    clusters = [c for c in clusters if len(c) != 1]
    scores = np.zeros((len(gold_clusters), len(clusters)))
    for i in range(len(gold_clusters)):
        for j in range(len(clusters)):
            scores[i, j] = phi4(gold_clusters[i], clusters[j])
    matching = linear_assignment(-scores)
    similarity = sum(scores[matching[0], matching[1]])

    # similarity = sum(scores[matching[:, 0], matching[:, 1]])
    return similarity, len(clusters), similarity, len(gold_clusters)

In [ ]:
for i, instance in enumerate(dl):
    outputs = model.pred_with_labels(**instance)
    break

In [ ]:
! free -h

In [ ]:
instance['coref'].keys(), outputs['coref'].keys(), outputs.keys(), outputs['coref']['eval'].keys()

In [ ]:
print('=None, '.join(['clusters', 'gold_clusters', 'mention_to_predicted', 'mention_to_gold']))

In [ ]:
ll =  outputs['coref']['eval']

In [ ]:
ceafe(ll['clusters'], ll['gold_clusters'])

In [ ]:
phi4(ll['clusters'], ll['gold_clusters'])

In [ ]:
muc(ll['clusters'], ll['mention_to_gold'])

In [ ]:
b_cubed(ll['clusters'], ll['mention_to_gold'])